In [6]:
!pip install tsfresh

In [2]:
!pip install sktime

     |████████████████████████████████| 5.6MB 4.7MB/s 
     |████████████████████████████████| 9.5MB 26.2MB/s 
     |████████████████████████████████| 6.8MB 46.3MB/s 
  Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [3]:
import sktime
import numpy as np
import pandas as pd

Loading fNIRS Data

In [9]:
fnirs_namesCol = ['Subject Name', 'Event Name', 'Channel Name', 'Start time', 'End time']

for q in range(131):
  strVar = 'v' + str(q)
  fnirs_namesCol.append(strVar)

fnirs_df = pd.read_csv('/content/drive/MyDrive/CS539/211_AXCPT19_TK_axcaxwr_hb_cs539.csv', header = 0, names = fnirs_namesCol)
fnirs_col_name = list(fnirs_df.columns)
fnirs_trans_df = pd.DataFrame(data = fnirs_df, columns = fnirs_col_name)

In [ ]:
fnirs_trans_df.head()

In [19]:
from sklearn.model_selection import train_test_split

fnirs_X_df = fnirs_trans_df.drop(['Event Name'], axis=1)
fnirs_Y_sr = fnirs_trans_df['Event Name']
fnirs_X_train, fnirs_X_test, fnirs_Y_train, fnirs_Y_test = train_test_split(fnirs_X_df,fnirs_Y_sr)

In [20]:
print(fnirs_X_train.shape, fnirs_X_test.shape, fnirs_Y_train.shape, fnirs_Y_test.shape)

(1380, 135) (460, 135) (1380,) (460,)


In [22]:
# encoding labels in X datasets
from sklearn import preprocessing

cat_cols = ['Subject Name', 'Channel Name', 'Start time', 'End time']
# cat_cols = ['group', 'trial']
enc = preprocessing.LabelEncoder()

for col in cat_cols:
    fnirs_X_train[col] = fnirs_X_train[col].astype('str')
    fnirs_X_test[col] = fnirs_X_test[col].astype('str')
    fnirs_X_train[col] = enc.fit_transform(fnirs_X_train[col])
    fnirs_X_test[col] = enc.transform(fnirs_X_test[col]) # this throws a labeling error, I assume because of the subject IDs

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [29]:
#isolate the time series
fnirs_X_train_timedata = fnirs_X_train[fnirs_X_train.columns[4:135]]
fnirs_X_test_timedata = fnirs_X_test[fnirs_X_test.columns[4:135]]



fnirs_X_train_timedata['combine'] = fnirs_X_train_timedata.values.tolist()
fnirs_X_test_timedata['combine'] = fnirs_X_test_timedata.values.tolist()
fnirs_X_train_timedata = fnirs_X_train_timedata['combine']
fnirs_X_test_timedata = fnirs_X_test_timedata['combine']

#convert to dataframe
fnirs_X_train_timedata = fnirs_X_train_timedata.to_frame()
fnirs_X_test_timedata = fnirs_X_test_timedata.to_frame()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [30]:
fnirs_ts_train = pd.Series(fnirs_X_train_timedata['combine'].values, index=fnirs_X_train_timedata.index)
fnirs_X_ts_train = fnirs_ts_train.to_frame()

fnirs_ts_test = pd.Series(fnirs_X_test_timedata['combine'].values, index=fnirs_X_test_timedata.index)
fnirs_X_ts_test = fnirs_ts_test.to_frame()

In [31]:
for row_num in range(0,fnirs_X_ts_train.shape[0]):
  fnirs_train_series = pd.Series(fnirs_X_ts_train.iat[row_num,0])
  fnirs_X_ts_train.iat[row_num,0] = fnirs_train_series

for row_num in range(0,fnirs_X_ts_test.shape[0]):
  fnirs_test_series = pd.Series(fnirs_X_ts_test.iat[row_num,0])
  fnirs_X_ts_test.iat[row_num,0] = fnirs_test_series

Verifying the format of the data

In [32]:
fnirs_X_ts_test.head()

,0
55,0 -0.183769 1 -0.172173 2 -0.15876...
1835,0 -0.926401 1 -0.901414 2 -0.87556...
487,0 -0.712602 1 -0.738459 2 -0.75479...
1638,0 -0.170718 1 -0.173866 2 -0.17271...
1813,0 1.377929 1 1.422019 2 1.46723...


Proximity Forest

In [ ]:
from sktime.classification.distance_based import ProximityForest

prox_forest = ProximityForest(n_estimators=5)
prox_forest.fit(fnirs_X_ts_train, fnirs_Y_train)
prox_forest.score(fnirs_X_ts_test, fnirs_Y_test)

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
